In [247]:
import numpy as np
from scipy.optimize import minimize

from network_model import *
import time

np.random.seed(23)

from cptopt.optimizer import MeanVarianceFrontierOptimizer
from cptopt.utility import CPTUtility

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams.update(plt.rcParamsDefault)
plt.rcParams.update({"font.size" : 15,
                     "figure.dpi" : 100, 
                     "grid.alpha" : 0.3,
                     "axes.grid": True,
                     "axes.axisbelow" : True,
                     "figure.figsize":(8,6),
                     "mathtext.fontset":"cm",
                     "xtick.labelsize": 14,
                     "ytick.labelsize": 14,
                     "axes.labelsize": 16, 
                     "legend.fontsize": 13.5})

USE_TEX = False
if USE_TEX:
    plt.rc("text", usetex=True)
    plt.rc("text.latex", preamble=r"""
     \usepackage{times}
     \usepackage{mathptmx}""")
else:
    plt.rc("text", usetex=False)
plt.rc("font", family="serif")

In [ ]:
solver = []

timea = time.time()

for _ in range(200):

    # generate gamble returns
    gambles = generate_gambles(N=5, left=1.3, right=1.4)
    gambles.append({"outcomes":[1.1,0.0], "probs":[1,0]})    
    samples = 100
    gamble_returns = np.zeros((samples,len(gambles)))
    for i,g in enumerate(gambles):
        gamble_returns[:,i] = np.random.choice(g["outcomes"], samples, p=g["probs"]) - 1

    # define CPT utility function
    utility = CPTUtility(
        gamma_pos=8.4, gamma_neg=11.4,
        delta_pos=0.77, delta_neg=0.79
    )

    # optimization
    mv = MeanVarianceFrontierOptimizer(utility)
    mv.optimize(gamble_returns)
    solver.append(mv._best_util)
        
print(time.time() - timea)
        
with open("solver.pickle", 'wb') as f:
    pickle.dump(solver, f)

In [ ]:
no_solver = []

timea = time.time()

for _ in range(200):

    # generate gamble returns
    gambles = generate_gambles(N=5, left=1.3, right=1.4)
    gambles.append({"outcomes":[1.1,0.0], "probs":[1,0]})    
    samples = 100
    gamble_returns = np.zeros((samples,len(gambles)))
    for i,g in enumerate(gambles):
        gamble_returns[:,i] = np.random.choice(g["outcomes"], samples, p=g["probs"]) - 1

    # define CPT utility function
    utility = CPTUtility(
        gamma_pos=8.4, gamma_neg=11.4,
        delta_pos=0.77, delta_neg=0.79
    )

    # optimization
    mv = MeanVarianceFrontierOptimizer(utility)
    mv.optimize(gamble_returns)
    no_solver.append(mv._best_util)
        
print(time.time() - timea)
        
with open("no_solver.pickle", 'wb') as f:
    pickle.dump(no_solver, f)

In [ ]:
with open("solver.pickle", "rb") as f:
    solver = pickle.load(f)

plt.plot(solver, label="solver")
plt.plot(no_solver, label="no solver")
plt.legend()
plt.show()

In [396]:
N = 4

# generate gamble returns
gambles = generate_gambles(N=N, left=1.3, right=1.4)
gambles.append({"outcomes":[1.1,0.0], "probs":[1,0]})    
samples = 1000
gamble_returns = np.zeros((samples,len(gambles)))
for i,g in enumerate(gambles):
    gamble_returns[:,i] = np.random.choice(g["outcomes"], samples, p=g["probs"]) - 1

In [397]:
np.mean(gamble_returns, axis=0).round(3)

array([0.106, 0.148, 0.102, 0.164, 0.1  ])

In [398]:
def portfolio_variance(x, cov):
    return x.T @ cov @ x

def portfolio_expected_value(x, mu):
    return - mu @ x

In [399]:
COV = np.cov(gamble_returns, rowvar=False)
MU = np.mean(gamble_returns, axis=0)

In [400]:
x0 = np.repeat(1/(N+1), N+1)
cons = ({'type': 'eq', 'fun': lambda x:  sum(x) - 1})
bnds = [(0, 1)] * (N+1)

In [401]:
result = minimize(portfolio_variance, 
                  x0=x0, 
                  args=(COV), 
                  constraints=cons,
                  bounds=bnds)

u_min = result.fun

In [402]:
result = minimize(portfolio_expected_value, 
                  x0=x0, 
                  args=(MU), 
                  constraints=cons,
                  bounds=bnds)

u_max = - result.fun

In [403]:
u_min, u_max

(4.8884587560642804e-08, 0.16430140454854394)

In [404]:
# define CPT utility function
utility = CPTUtility(
    gamma_pos=8.4, gamma_neg=11.4,
    delta_pos=0.77, delta_neg=0.79
)

N = len(gamble_returns)
p_weights = utility.cumulative_weights(N, delta=utility.delta_pos)
n_weights = utility.cumulative_weights(N, delta=utility.delta_neg)

In [405]:
def cpt_utility(x, bnds, x0, COV):
        
    cons = ({'type': 'eq', 'fun': lambda y:  sum(y) - 1},
            {'type': 'ineq', 'fun': lambda y: MU @ y - x[0]})
        
    res = minimize(portfolio_variance,
                   x0=x0,
                   args=(COV),
                   constraints=cons,
                   bounds=bnds,
                   method="SLSQP")
    
    return - utility.evaluate(res.x, gamble_returns, p_weights, n_weights)[0]

In [408]:
delta = u_max - u_min
out = minimize(cpt_utility, 
               x0=(u_min + 0.7 * delta), 
               bounds=((u_min, u_max),),
               args=(bnds, x0, COV),
               method="SLSQP",
               tol=1e-4)

In [410]:
cons = ({'type': 'eq', 'fun': lambda y:  sum(y) - 1},
        {'type': 'ineq', 'fun': lambda y: MU @ y - out.x[0]})

res = minimize(portfolio_variance,
               x0=x0,
               args=(COV),
               constraints=cons,
               bounds=bnds,
               method="SLSQP")

In [413]:
res.x

array([0.03420618, 0.12714739, 0.00844075, 0.19865397, 0.63155171])